In [38]:
import os
import csv
import gzip
import io
import pprint
import collections
import operator

import pandas
import requests

In [4]:
# Download all data from BindingDB
# !wget --directory-prefix download https://www.bindingdb.org/bind/downloads/BindingDB_All_2015m3.tsv.zip
# !unzip -d download download/BindingDB_All_2015m3.tsv.zip
# !rm download/BindingDB_All_2015m3.tsv.zip
# !mv download/BindingDB_All.tsv download/BindingDB_All_2015m3.tsv
# !gzip download/BindingDB_All_2015m3.tsv
!shasum download/BindingDB_All_2015m3.tsv.gz

6f802ba65538d4e28a1c0be7d20ac751f645d8c3  download/BindingDB_All_2015m3.tsv.gz


In [39]:
target_fields = [
    'BindingDB Target Chain  Sequence',
    'PDB ID(s) of Target Chain',
    'UniProt (SwissProt) Recommended Name of Target Chain',
    'UniProt (SwissProt) Entry Name of Target Chain',
    'UniProt (SwissProt) Primary ID of Target Chain',
    'UniProt (SwissProt) Secondary ID(s) of Target Chain',
    'UniProt (SwissProt) Alternative ID(s) of Target Chain',
    'UniProt (TrEMBL) Submitted Name of Target Chain',
    'UniProt (TrEMBL) Entry Name of Target Chain',
    'UniProt (TrEMBL) Primary ID of Target Chain',
    'UniProt (TrEMBL) Secondary ID(s) of Target Chain',
    'UniProt (TrEMBL) Alternative ID(s) of Target Chain',
]

chains_key = 'Number of Protein Chains in Target (>1 implies a multichain complex)'

def read_bindingdb(path):
    """
    Field documentation: https://www.bindingdb.org/bind/chemsearch/marvin/BindingDB-TSV-Format.pdf
    """
    read_file = gzip.open(path, 'rb')
    text = io.TextIOWrapper(read_file)
    reader = csv.reader(text, delimiter='\t')
    header = next(reader)
    chains_index = header.index(chains_key)
    taget0_index = chains_index + 1
    ligand_fields = header[:chains_index + 1]
    for row in reader:
        row = [x if x else None for x in row]
        ligand_values = row[:chains_index + 1]
        rowdict = collections.OrderedDict(zip(ligand_fields, ligand_values))
        for key in [chains_key]:
            rowdict[key] = int(rowdict[key])
        chains = list()
        assert rowdict[chains_key] == len(row[taget0_index:]) / len(target_fields)
        for i in range(rowdict[chains_key]):
            i_0 = taget0_index + i * len(target_fields)
            i_1 = taget0_index + (i + 1) * len(target_fields)
            target_values = row[i_0:i_1]
            chain = collections.OrderedDict(zip(target_fields, target_values))
            chains.append(chain)
        rowdict['chains'] = chains
        yield rowdict
    read_file.close()

In [18]:
# uniprot to entrez gene mapping
url = 'http://git.dhimmel.com/uniprot/data/map/GeneID.tsv.gz'
response = requests.get('http://git.dhimmel.com/uniprot/data/map/GeneID.tsv.gz', stream=True)
text = gzip.GzipFile(fileobj=response.raw)
text = io.TextIOWrapper(text)
uniprot_df = pandas.read_table(text, engine='python')

In [34]:
uniprot_to_entrez = dict()
for uniprot, entrez in zip(uniprot_df.uniprot, uniprot_df.GeneID):
    uniprot_to_entrez.setdefault(uniprot, set()).add(str(entrez))

In [47]:
path = os.path.join('download', 'BindingDB_All_2015m3.tsv.gz')
bindingdb_generator = read_bindingdb(path)

bindings = list()
for i, row in enumerate(bindingdb_generator):
    #if i > 10000:
    #    break
    if len(row['chains']) != 1:
        continue
    chain, = row['chains']
    uniprots = chain['UniProt (SwissProt) Primary ID of Target Chain']
    if not uniprots:
        continue
    uniprots = uniprots.split(',')

    template = dict()
    template['bindingdb_id'] = row['BindingDB MonomerID']
    template['reaction_id'] = row['BindingDB Reactant_set_id']
    template['source'] = row['Curation/DataSource']
    template['organism'] = row['Target Source Organism According to Curator or DataSource']
    template['pubmed'] = row['PMID']
    template['doi'] = row['Article DOI']

    affinities = {'Ki': row['Ki (nM)'], 'Kd': row['Kd (nM)'], 'IC50': row['IC50 (nM)']}
    for measure, affinity in affinities.items():
        if affinity is None:
            continue
        for uniprot in uniprots:
            entrez_set = uniprot_to_entrez.get(uniprot)
            if not entrez_set:
                # uniprot_id not found in mapping
                continue
            for entrez in entrez_set:
                binding = template.copy()
                binding['measure'] = measure
                binding['affinity_nM'] = affinity
                binding['uniprot'] = uniprot
                binding['entrez_gene'] = entrez
                bindings.append(binding)


In [56]:
lt, gt, eq = 0, 0, 0
for binding in bindings:
    affinity = binding['affinity_nM']
    if affinity.startswith('<'):
        affinity = affinity.lstrip('<')
        affinity = float(affinity)
        if affinity >= 10.0:
            affinity -= 1.0
        lt += 1
    elif affinity.startswith('>'):
        affinity = affinity.lstrip('>')
        affinity = float(affinity)
        affinity += 1.0
        gt += 1
    else:
        affinity = float(affinity)
        eq += 1
    binding['affinity_nM'] = affinity
print(lt, gt, eq)

954 137761 631165


In [57]:
fields = ['reaction_id', 'bindingdb_id', 'uniprot', 'entrez_gene',
          'measure', 'affinity_nM', 'source', 'organism', 'pubmed', 'doi']
with gzip.open('data/binding.tsv.gz', 'wb') as write_file:
    wrapper = io.TextIOWrapper(write_file)
    writer = csv.DictWriter(wrapper, delimiter='\t', fieldnames=fields)
    writer.writeheader()
    bindings.sort(key=operator.itemgetter(*fields))
    writer.writerows(bindings)

In [11]:
path = os.path.join('download', 'BindingDB_All_2015m3.tsv.gz')
bindingdb_generator = read_bindingdb(path)

measure_keys = ['Ki (nM)', 'IC50 (nM)', 'Kd (nM)', 'EC50 (nM)'] #, 'kon (M-1-s-1)', 'koff (s-1)']

measures = list()
for i, row in enumerate(bindingdb_generator):
    #if i > 10000:
    #    break
    if len(row['chains']) != 1:
        continue
    chain, = row['chains']
    uniprot = chain['UniProt (SwissProt) Primary ID of Target Chain']
    if not uniprot:
        continue
    measure_set = frozenset(key for key in measure_keys if row[key] is not None)
    measures.append(measure_set)

pprint.pprint(collections.Counter(measures))

{frozenset(): 881,
 frozenset({'Ki (nM)'}): 245475,
 frozenset({'Kd (nM)', 'Ki (nM)'}): 4,
 frozenset({'Kd (nM)', 'EC50 (nM)'}): 8,
 frozenset({'IC50 (nM)', 'Ki (nM)'}): 985,
 frozenset({'EC50 (nM)'}): 74549,
 frozenset({'Kd (nM)'}): 55044,
 frozenset({'IC50 (nM)'}): 480070,
 frozenset({'Kd (nM)', 'IC50 (nM)'}): 76,
 frozenset({'EC50 (nM)', 'IC50 (nM)'}): 631,
 frozenset({'EC50 (nM)', 'Ki (nM)'}): 574,
 frozenset({'IC50 (nM)', 'EC50 (nM)', 'Ki (nM)'}): 23,
 frozenset({'IC50 (nM)', 'Kd (nM)', 'Ki (nM)'}): 1}


In [16]:
# Number of chains (proteins in target)
path = os.path.join('download', 'BindingDB_All_2015m3.tsv.gz')
bindingdb_generator = read_bindingdb(path)
collections.Counter(int(row[chains_key]) for row in bindingdb_generator)

Counter({1: 1073428, 2: 35637, 3: 5398, 4: 527, 6: 343, 5: 302, 12: 3, 19: 1})

In [25]:
# Targets that mapped to SwissProt
path = os.path.join('download', 'BindingDB_All_2015m3.tsv.gz')
bindingdb_generator = read_bindingdb(path)

collections.Counter(
    bool(row['chains'][0]['UniProt (SwissProt) Primary ID of Target Chain'])
    for row in bindingdb_generator if len(row['chains']) == 1
)

Counter({True: 858321, False: 215107})

In [26]:
# Species
path = os.path.join('download', 'BindingDB_All_2015m3.tsv.gz')
bindingdb_generator = read_bindingdb(path)

collections.Counter(
    row['Target Source Organism According to Curator or DataSource']
    for row in bindingdb_generator if
    len(row['chains']) == 1 and 
    row['chains'][0]['UniProt (SwissProt) Primary ID of Target Chain']
)

Counter({'Homo sapiens': 673750, 'Rattus norvegicus': 95095, 'Mus musculus': 26498, 'Bos taurus': 14064, 'Cavia porcellus': 8565, 'Sus scrofa': 4372, 'Oryctolagus cuniculus': 4262, 'Electrophorus electricus': 2784, 'Caenorhabditis elegans': 2245, 'Ovis aries': 2138, 'Equus caballus': 1720, 'Saccharomyces cerevisiae (strain ATCC 204508 / S288c)': 1383, 'Trypanosoma cruzi': 1222, 'Toxoplasma gondii': 1205, 'Plasmodium falciparum (isolate K1 / Thailand)': 966, 'Bacillus anthracis': 916, 'Pneumocystis carinii': 889, 'Gallus gallus': 875, 'Plasmodium falciparum': 819, 'Agaricus bisporus': 813, 'Plasmodium falciparum (isolate FcB1 / Columbia)': 726, 'Lactobacillus casei': 701, 'Human herpesvirus 1 (strain 17)': 680, 'Escherichia coli': 675, 'Staphylococcus aureus': 616, 'Torpedo californica': 574, 'Candida albicans': 491, None: 474, 'Clostridium botulinum': 427, 'Human cytomegalovirus (strain AD169)': 375, 'Cryptosporidium parvum': 355, 'Canis familiaris': 312, 'Enterobacter cloacae': 307, '